<!-- Assignment 3 - WS 2023 -->

# Convolutional Neural Networks (22 points)

This notebook contains the third assignment for the exercises in Deep Learning and Neural Nets 1.
It provides a skeleton, i.e. code with gaps, that will be filled out by you in different exercises.
All exercise descriptions are visually annotated by a vertical bar on the left and some extra indentation,
unless you already messed with your jupyter notebook configuration.
Any questions that are not part of the exercise statement do not need to be answered,
but should rather be interpreted as triggers to guide your thought process.

**Note**: The cells in the introductory part (before the first subtitle)
perform all necessary imports and provide utility function that should work without problems.
Please, do not alter this code or add extra import statements in your submission, unless it is explicitly requested!

<span style="color:#d95c4c">**IMPORTANT:**</span> Please, change the name of your submission file so that it contains your student ID!

In this assignment, the goal is to get familiar with **Convolutional Neural Networks**. Essentially, a CNN is a multi-layer perceptron that uses convolutional instead of fully connected layers. Since convolutions are known to be useful for image processing, CNNs have become a powerful tool for learning features from images. However, they have proven to beat alternative architectures in a variety of other domains.

In [184]:
import numpy as np

from nnumpy import Module
from nnumpy.utils import sig2col
from nnumpy.testing import gradient_check

rng = np.random.default_rng(1856)

## Convolution

The main difference of CNNs with the fully connected networks we tackled thus far, is the *convolution operation*. 

###### The Math

Mathematically, a (discrete) convolution operates on two functions, so that

$$(f * g)[n] = \sum_{k \in \mathbb{Z}} f[k] g[n - k].$$

For image processing, the discrete functions $f$ and $g$ and replaced by images. After all, an image can be considered a function of pixel indices to pixel intensities. Also, images have (at least) two dimensions: width and height. Therefore, if we represent images as matrices of pixel intensities, we can write the convolution of an image $\boldsymbol{X} \in \mathbb{R}^{H \times W}$ with a so-called *kernel* $\boldsymbol{K} \in \mathbb{R}^{R_1 \times R_2}$ as follows:

$$(\boldsymbol{K} * \boldsymbol{X})_{a,b} = \sum_{i=1}^{R_1} \sum_{j=1}^{R_2} k_{i,j} x_{a - i + 1,b - j + 1}.$$

Instead of using the actual convolution operation, convolutional layers are often implemented as the *cross-correlation* of kernel and image instead:

$$(\boldsymbol{K} \star \boldsymbol{X})_{a,b} = \sum_{i=1}^{R_1} \sum_{j=1}^{R_2} k_{i,j} x_{a + i - 1,b + j - 1}.$$

It might be useful to note that unlike the convolution, the cross-correlation is not commutative, i.e. $\boldsymbol{K} \star \boldsymbol{X} \neq \boldsymbol{X} \star \boldsymbol{K}$, whereas $\boldsymbol{K} * \boldsymbol{X} = \boldsymbol{X} * \boldsymbol{K}$.

### Exercise 1: Cross-correlation vs Convolution (3 Points)

Implementation-wise, there is little difference between cross-correlation and convolution. It is even quite straightforward to implement one, given an implementation of the other. To keep things simple, this exercise is limited to the one-dimensional variants of these operations (for now). How hard would it be to make your implementation of the convolution function commutative?

> Implement functions to compute the cross-correlations and convolutions of one-dimensional signals. Obviously, you should **not** use functions like `np.convolve` or `np.correlate`.

In [185]:
def cross_correlation1d(x, k):
    """
    Compute a one-dimensional cross-correlation.
    
    Parameters
    ----------
    x : (L, ) ndarray
        Input data for the cross-correlation.
    k : (R, ) ndarray
        Kernel weights for the cross-correlation.
        
    Returns
    -------
    features : (L') ndarray
        Cross-correlation of the input data with the kernel.
    """
    # YOUR CODE HERE
    
    L = x.shape[0]
    R = k.shape[0]

    L_prime = L - R + 1

    features = np.zeros(L_prime)

    for i in range(L_prime): # forall a in L_prime
        for j in range(R):
            features[i] += x[i + j] * k[j]
    
    return features


def convolution1d(x, k):
    """
    Compute a one-dimensional convolution.
    
    Parameters
    ----------
    x : (L, ) ndarray
        Input data for the convolution.
    k : (R, ) ndarray
        Kernel weights for the convolution.
        
    Returns
    -------
    features : (L', ) ndarray
        Result of convolving the input data with the kernel.
    """
    # YOUR CODE HERE
    L = x.shape[0]
    R = k.shape[0]

    L_prime = L - R + 1

    features = np.zeros(L_prime)

    for i in range(L_prime): # equivalent to the "forall a in L_prime"
        for j in range(R):
            features[i] += x[i - j] * k[j]
    
    return features



In [186]:
# Test Cell: do not edit or delete!
x = rng.standard_normal(11)
k = rng.standard_normal(3)
corr = cross_correlation1d(x, k)
assert isinstance(corr, np.ndarray), (
    "ex1: output of cross_correlation1d is not a numpy array"
)
assert corr.size == 9, (
    "ex1: output of cross_correlation1d has incorrect size"
)

In [187]:
# Test Cell: do not edit or delete!

In [188]:
# Test Cell: do not edit or delete!
x = rng.standard_normal(11)
k = rng.standard_normal(3)
conv = convolution1d(x, k)
assert isinstance(conv, np.ndarray), (
    "ex1: output of convolution1d is not a numpy array"
)
assert conv.size == 9, (
    "ex1: output of convolution1d has incorrect size"
)

In [189]:
# Test Cell: do not edit or delete!

###### The Code

This direct implementation does not offer a lot of features. For starters, it does not provide functionality to process multiple samples at once. Furthermore, practical implementations of convolutional layers normally require support for *channels*. After all, it is common practice to create multiple feature maps from a single signal to compensate for the spatial reduction through pooling and strides. These features can be incorporated in the mathematical formulation as follows:
$$(\boldsymbol{K} \star \boldsymbol{X})_{n,c_\mathrm{out},a,b} = \sum_{c_\mathrm{in}=1}^{C_\mathrm{in}} \sum_{i=1}^{R_1} \sum_{j=1}^{R_2} k_{c_\mathrm{out},c_\mathrm{in},i,j} x_{n,c_\mathrm{in},a + i - 1,b + j - 1}.$$

Of course this makes things a bit more complicated. It also introduces an extra loop over the number of input channels. In order to implement the above formula efficiently, we can use a trick that is commonly referred to as `im2col`. The idea of `im2col` is to represent the input tensor ($\in \mathbb{R}^{N \times C_\mathrm{in} \times A \times B}$) by a tensor in $\mathbb{R}^{N \times A' \times B' \times (C_\mathrm{in} \cdot R_1 \cdot R_2)}$ where each "column" holds the elements in the window of the convolution. This allows the convolution to be computed as a simple matrix product with the (reshaped) kernel matrix $\boldsymbol{K} \in \mathbb{R}^{C_\mathrm{out} \times (C_\mathrm{in} \cdot R_1 \cdot R_2)}$, i.e.

$$(\boldsymbol{K} \star \boldsymbol{X})_{n,c_\mathrm{out},a,b} = \sum_{r=1}^{C_\mathrm{in} \cdot R_1 \cdot R_2} x_{n,a,b,r} k_{r,c_\mathrm{out}}.$$

This trick is (efficiently) implemented in the `sig2col` function (slightly different name, since the function allows for modalities other than images). It takes **two inputs**: the signal to be convolved and the shape of the kernel as a tuple.

In [190]:
# sig2col on 1D signal
x = np.arange(7)
kernel_shape = (3, )
sig2col(x, kernel_shape)

array([[0, 1, 2],
       [1, 2, 3],
       [2, 3, 4],
       [3, 4, 5],
       [4, 5, 6]])

In [191]:
# image
im = np.arange(16).reshape(4, 4)
im

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [192]:
# 3x2 windows in image as vectors
kernel_shape = (3, 2)
sig2col(im, (kernel_shape)).reshape(-1, 3 * 2)

array([[ 0,  1,  4,  5,  8,  9],
       [ 1,  2,  5,  6,  9, 10],
       [ 2,  3,  6,  7, 10, 11],
       [ 4,  5,  8,  9, 12, 13],
       [ 5,  6,  9, 10, 13, 14],
       [ 6,  7, 10, 11, 14, 15]])

### Exercise 2: Multi-channel Convolutions (4 Points)

Time to implement an actually practical convolution function that can handle multiple channels. Let us make it a 2D convolution at once.

 > Implement the `multi_channel_convolution2d` function below. You can use the `sig2col` function to implement the convolution by means of a dot product.
 
**Hint:** When using the `sig2col` function, you might need to fiddle with the order of dimensions of your numpy arrays to align everything properly.

In [193]:
def multi_channel_convolution2d(x, k):
    """
    Compute the multi-channel convolution of multiple samples.
    
    Parameters
    ----------
    x : (N, Ci, A, B)
    k : (Co, Ci, R1, R2)
    
    Returns
    -------
    y : (N, Co, A', B')
    
    See Also
    --------
    sig2col : can be used to convert (N, Ci, A, B) ndarray 
              to (N, Ci, A', B', R1, R2) ndarray.
    """
    # YOUR CODE HERE
    
    N, _, _, _ = x.shape
    Co, _, R1, R2 = k.shape

    transformed = sig2col(x, (R1, R2)) # shape is N, Ci, A', B', R1, R2
    transformed = transformed.transpose(0, 2, 3, 1, 4, 5) # shape is N, A', B', Ci, R1, R2

    # now we multiply Ci, R1, R2 together
    A_prime = transformed.shape[1]
    B_prime = transformed.shape[2]

    transformed = transformed.reshape(N, A_prime, B_prime, -1) # shape is now N, A', B', Ci * R1 * R2

    k = k.reshape(-1, Co) # shape is Ci * R1 * R2, Co

    out = transformed @ k # shape is now N, A', B', Co
    out = out.transpose(0, 3, 1, 2) # shape is now N, Co, A', B'

    return out  



In [194]:
# Test Cell: do not edit or delete!
x = rng.standard_normal(size=(10, 1, 28, 28))
k = rng.standard_normal(size=(5, 1, 3, 3))
s = multi_channel_convolution2d(x, k)
assert isinstance(s, np.ndarray), (
    "ex2: output of multi_channel_convolution2d is not a numpy array"
)
assert s.shape == (x.shape[0], k.shape[0], 26, 26), (
    "ex2: output of multi_channel_convolution2d has incorrect shape"
)

In [195]:
# Test Cell: do not edit or delete!

###### The Module

The multi-channel convolution pretty much covers the forward pass for a typical convolutional layer. For the backward pass, we will need the gradients of this operations. In the case of the simple convolution from the first exercise, it can easily be derived that the gradients w.r.t. inputs and weights are again convolutions, since

$$\begin{aligned}
    \frac{\partial L}{\partial w_i} & = \sum_a \frac{\partial L}{\partial s_a} \frac{\partial s_a}{\partial w_i} = \sum_a \delta_a x_{i+a} \\
    \frac{\partial L}{\partial x_i} & = \sum_a \frac{\partial L}{\partial s_a} \frac{\partial s_a}{\partial x_i} = \sum_{a'} w_{a'} \delta_{i-a'},
\end{aligned}$$

where

$$\begin{aligned}
    \frac{\partial s_a}{\partial w_i} & = \frac{\partial}{\partial w_i} \left( \sum_r w_r x_{a+r} \right) = x_{a+i} \\
    \frac{\partial s_a}{\partial x_i} & = \frac{\partial}{\partial x_i} \left( \sum_r w_r x_{a+r} \right) = w_{i - a}.
\end{aligned}$$

Fortunately, this approach generalises to multi-channel convolutions. For the convolution of a 1D signal with $c_\mathrm{i}$ channels so that the output has $c_\mathrm{o}$ channels, it can be verified that

$$\begin{aligned}
    \frac{\partial L}{\partial w_{c_\mathrm{o}, c_\mathrm{i}, i}} & = \sum_a \frac{\partial L}{\partial s_{c_\mathrm{o},a}} \frac{\partial s_{c_\mathrm{o},a}}{\partial w_{c_\mathrm{o}, c_\mathrm{i}, i}} = \sum_a \delta_{c_\mathrm{o},a} x_{c_\mathrm{i},i+a} \\
    \frac{\partial L}{\partial x_{c_\mathrm{i}, i}} & = \sum_{c_\mathrm{o}} \sum_a \frac{\partial L}{\partial s_{c_\mathrm{o},a}} \frac{\partial s_{c_\mathrm{o},a}}{\partial x_{c_\mathrm{i}, i}} = \sum_{c_\mathrm{o}} \sum_{a'} w_{c_\mathrm{o}, c_\mathrm{i}, a'} \delta_{c_\mathrm{o}, i-a'},
\end{aligned}$$

where

$$\begin{aligned}
    \frac{\partial s_{c_\mathrm{o},a}}{\partial w_{c_\mathrm{o}, c, i}} & = \frac{\partial}{\partial w_{c_\mathrm{o}, c, i}} \left( \sum_{c_\mathrm{i}} \sum_r w_{c_\mathrm{o}, c_\mathrm{i}, r} x_{c_\mathrm{i},a+r} \right) = x_{c, a+i} \\
    \frac{\partial s_{c_1,a}}{\partial x_{c_2, i}} & = \frac{\partial}{\partial x_{c_2,i}} \left( \sum_{c_\mathrm{i}} \sum_r w_{c_\mathrm{o}, c_\mathrm{i}, r} x_{c_\mathrm{i}, a+r} \right) = w_{c_1, c_2, i - a}.
\end{aligned}$$

We can conclude that the gradients of multi-channel convolutions can again be expressed as multi-channel convolutions - taking into account that we compute the convolutions for multiple samples at once.

### Exercise 3: Convolutional Layer (7 Points)

Now, you should be able to implement both forward and backward pass in a module. Have you already thought about the shape of the bias parameter?

 > Implement the `Conv2D` module below. You can use the `multi_channel_convolution2d` function from the previous exercise to implement forward and backward pass.

In [196]:
class Conv2d(Module):
    """ Numpy DL implementation of a 2D convolutional layer. """
    
    def __init__(self, in_channels, out_channels, kernel_size, use_bias=True):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.use_bias = use_bias
        
        # create parameters 'w' and 'b'
        # YOUR CODE HERE
        
        if self.use_bias:
            bias_shape = (1, out_channels, 1, 1)
            self.register_parameter('b', np.empty(bias_shape)) # empty as we call reset_parameters later
        self.register_parameter('w', np.empty((out_channels, in_channels, *kernel_size)))
        
        self.reset_parameters()
        
    def reset_parameters(self, seed: int = None):
        """ 
        Reset the parameters to some random values.
        
        Parameters
        ----------
        seed : int, optional
            Seed for random initialisation.
        """
        rng = np.random.default_rng(seed)
        self.w = rng.standard_normal(size=self.w.shape)
        if self.use_bias:
            self.b = np.zeros_like(self.b)
        
    def compute_outputs(self, x):
        """
        Parameters
        ----------
        x : (N, Ci, H, W) ndarray
        
        Returns
        -------
        feature_maps : (N, Co, H', W') ndarray
        cache : ndarray or tuple of ndarrays
        """
        # YOUR CODE HERE
        
        output = multi_channel_convolution2d(x, self.w)
        if self.use_bias:
            output += self.b
        
        return output, x
        
        
    
    def compute_grads(self, grads, cache):
        """
        Parameters
        ----------
        grads : (N, Co, H', W') ndarray
        cache : ndarray or tuple of ndarrays
        
        Returns
        -------
        dx : (N, Ci, H, W) ndarray
        """
        # YOUR CODE HERE

        # The following is now based on the slides at page 26-48

        # dx = None

        # padding the grads (these are the deltas)      

        N, Co, H_prime, W_prime = grads.shape
        print(f'N = {N}, Co = {Co}, H_prime = {H_prime}, W_prime = {W_prime}')

        px, py = self.kernel_size
        px = px - 1
        py = py - 1

        padded_grads = np.pad(grads, ((0, 0), (0, 0), (px, px), (py, py)), mode='constant', constant_values=0) # no padding in axis 0, 1; constant padding in axis 2, 3

        # flipped_w = self.w.transpose(1, 0, 2, 3) # flip along the first diagonal (lecture 15.11. at the end)
        flipped_w = np.flip(self.w, axis=(2,3)).transpose(1,0,2,3)
        # flipped_w = np.flip(self.w, axis=(2,3))
        

        print(f'Shape of flipped_w {flipped_w.shape}')
        print(f'Shape of padded_grads {padded_grads.shape}')
        dx = multi_channel_convolution2d(padded_grads, flipped_w) # flip along the second diagonal
        dx = np.flip(np.flip(dx, axis=2), axis=3)
        # dx has now the shape (N, Co, H', W')

        # right shape but still wrong values

        
        
        # Correct
        print(f'Shape of cache {cache.shape}')
        print(f'Shape of grads {grads.shape}')
        dw = multi_channel_convolution2d(cache.transpose(1, 0, 2, 3), grads.transpose(1, 0, 2, 3)).transpose(1, 0, 2, 3)
        self.w.grad = dw        
        

        # Correct
        if self.use_bias:
            self.b.grad = np.sum(grads, axis=(0,2,3)).reshape(self.b.shape)
                
        assert dx.shape == cache.shape, "Wrong shape of dx"
        assert self.w.grad.shape == self.w.shape, "Wrong shape of W.grad"
        assert self.b.grad.shape == self.b.shape, "Wrong shape of b.grad"

        return dx

In [197]:
# Test Cell: do not edit or delete!
conv = Conv2d(3, 8, (5, 3))
parameter_names = dict(conv.named_parameters())
assert "w" in parameter_names, (
    "ex3: Conv2d module does not have 'w' parameter"
)
assert "b" in parameter_names, (
    "ex3: Conv2d module does not have 'b' parameter"
)

In [198]:
# Test Cell: do not edit or delete!
x = rng.normal(size=(15, 3, 13, 13))
s, cache = conv.compute_outputs(x)
assert isinstance(s, np.ndarray), (
    "ex3: output of Conv2d.compute_outputs is not a numpy array"
)
assert s.shape == (len(x), conv.out_channels, 9, 11), (
    "ex3: output of Conv2d.compute_outputs has incorrect shape"
)

In [199]:
# Test Cell: do not edit or delete!
conv.zero_grad()
g = conv.compute_grads(np.ones_like(s), cache)
assert isinstance(g, np.ndarray), (
    "ex3: output of Conv2d.compute_grads is not a numpy array"
)
assert g.shape == x.shape, (
    "ex3: output of Conv2d.compute_grads has incorrect shape"
)

N = 15, Co = 8, H_prime = 9, W_prime = 11
Shape of flipped_w (3, 8, 5, 3)
Shape of padded_grads (15, 8, 17, 15)
Shape of cache (15, 3, 13, 13)
Shape of grads (15, 8, 9, 11)


In [200]:
# Test Cell: do not edit or delete!
assert np.nonzero(conv.w.grad), (
    "ex3: Conv2d.compute_grads does not compute gradients for 'w' parameter"
)
assert np.nonzero(conv.b.grad), (
    "ex3: Conv2d.compute_grads does not compute gradients for 'b' parameter"
)

In [201]:
# Test Cell: do not edit or delete!
assert gradient_check(conv, x, debug=True), (
    "ex3: Conv2d module does not pass gradient check"
)

N = 15, Co = 8, H_prime = 9, W_prime = 11
Shape of flipped_w (3, 8, 5, 3)
Shape of padded_grads (15, 8, 17, 15)
Shape of cache (15, 3, 13, 13)
Shape of grads (15, 8, 9, 11)
51.32921617910954 > 2e-06
dx numeric:  [[[[ 2.11691474e+00  1.94761395e+00 -5.00741302e+00 ...  1.36050033e+00
    -1.70434586e+00 -6.79629153e-01]
   [ 6.44994418e+00  2.49749070e+00  5.05847908e+00 ... -6.51061697e-01
     7.87579154e-01 -3.31968681e+00]
   [ 1.84335050e-01 -1.04309418e+01  4.52978952e-02 ... -4.80527598e+00
     6.03622524e+00  3.35047662e+00]
   ...
   [-1.30010739e+00 -1.02218526e+01  2.18738353e+00 ...  1.49302720e+01
    -4.30588975e+00  3.20219786e+00]
   [ 2.94529104e+00  4.41622464e+00 -6.80626346e+00 ...  1.26665448e+01
    -1.50886063e+00  1.33817082e-01]
   [-2.00361606e+00  3.11124256e+00  3.46469307e-01 ...  8.43659393e+00
     2.77895165e+00 -3.79179198e-01]]

  [[-3.05178690e+00  1.04381797e+01  8.12409382e-01 ... -4.11556158e-01
     3.71413427e+00 -1.96633218e+00]
   [-4.17306194e

AssertionError: ex3: Conv2d module does not pass gradient check

## Activation Functions

Although any activation function can be used in combination with convolutional neural networks, a very popular choice is the so-called *Rectified Linear Unit* (*ReLU*). The ReLU function maps all negative inputs to zero and all positive inputs to itself. Mathematically, this looks like

$$\mathrm{ReLU}(x) = \begin{cases} 0 & x \leq 0 \\ x & x > 0 \end{cases}.$$

An alternative activation function that is based on the ReLU, is the *Exponential Linear Unit* (*ELU*). Unlike the ReLU non-linearity, the ELU is able to keep the mean of the activations close to zero. It can be defined as follows:

$$\mathrm{ELU}(x \mathbin{;} \alpha) = \begin{cases} \alpha (e^x - 1) & x \leq 0 \\ x & x > 0 \end{cases}.$$

The parameter $\alpha$ in this non-linearity allows to specify the minimal negative value of the activations. Note that this $\alpha$ is a hyper-parameter that must be fixed before training, and is thus not learned.

### Exercise 4: Some Linear Units (3 Points)

A deep learning framework would not be complete without the ReLU and ELU activation functions. Time to add them!

 > Implement the `ReLU` and `ELU` activation function modules.

In [202]:
class ReLU(Module):
    """ NNumpy implementation of the Rectified Linear Unit. """
        
    def compute_outputs(self, s):
        """
        Parameters
        ----------
        s : (N, K) ndarray
        
        Returns
        -------
        a : (N, K) ndarray
        cache : ndarray or iterable of ndarrays
        """
        # YOUR CODE HERE
        
        a = np.maximum(0, s)
        return a, s

    
    def compute_grads(self, grads, cache):
        """
        Parameters
        ----------
        grads : (N, K) ndarray
        cache : ndarray or iterable of ndarrays

        Returns
        -------
        ds : (N, K) ndarrays
        """
        # YOUR CODE HERE
        return grads * (cache > 0)
        

class ELU(Module):
    """ NNumpy implementation of the Exponential Linear Unit. """
    
    def __init__(self, alpha=1.):
        super().__init__()
        if alpha < 0:
            raise ValueError("negative values for alpha are not allowed")
        
        self.alpha = alpha
        
        
    def compute_outputs(self, s):
        """
        Parameters
        ----------
        s : (N, K) ndarray
        
        Returns
        -------
        a : (N, K) ndarray
        cache : ndarray or iterable of ndarrays
        """
        # YOUR CODE HERE
        a = np.where(s > 0, s, self.alpha * (np.exp(s) - 1))
        return a, s
        
        

    
    def compute_grads(self, grads, cache):
        """
        Parameters
        ----------
        grads : (N, K) ndarray
        cache : ndarray or iterable of ndarrays

        Returns
        -------
        ds : (N, K) ndarrays
        """
        # YOUR CODE HERE
        return grads * np.where(cache > 0, 1, self.alpha * np.exp(cache))

In [203]:
# Test Cell: do not edit or delete!
s = np.linspace(-3, 3, 35).reshape(7, 5)
phi = ReLU()
a, cache = phi.compute_outputs(s)
assert isinstance(a, np.ndarray), (
    "ex4: output of ReLU.compute_outputs is not a numpy array"
)
assert a.shape == s.shape, (
    "ex4: output of ReLU.compute_outputs has incorrect shape"
)

In [204]:
# Test Cell: do not edit or delete!
g = phi.compute_grads(np.ones_like(s), cache)
assert isinstance(g, np.ndarray), (
    "ex4: output of ReLU.compute_grads is not a numpy array"
)
assert g.shape == s.shape, (
    "ex4: output of ReLU.compute_grads has incorrect shape"
)
assert gradient_check(phi, x, debug=True), (
    "ex4: ReLU module does not pass gradient check"
)

In [205]:
# Test Cell: do not edit or delete!
s = np.linspace(-3, 3, 35).reshape(7, 5)
phi = ELU()
a, cache = phi.compute_outputs(s)
assert isinstance(a, np.ndarray), (
    "ex4: output of ELU.compute_outputs is not a numpy array"
)
assert a.shape == s.shape, (
    "ex4: output of ELU.compute_outputs has incorrect shape"
)

In [206]:
# Test Cell: do not edit or delete!
g = phi.compute_grads(np.ones_like(s), cache)
assert isinstance(g, np.ndarray), (
    "ex4: output of ELU.compute_grads is not a numpy array"
)
assert g.shape == s.shape, (
    "ex4: output of ELU.compute_grads has incorrect shape"
)
assert gradient_check(phi, x, debug=True), (
    "ex4: ELU module does not pass gradient check"
)

## Spatial Reduction

The *weight sharing* in convolutional neural networks can drastically reduce the memory requirements for the weights. This effectively allows the input data to become larger, but since we need to store parts of the forward pass for back-propagation, the gains are rather limited. Of course, standard convolutions reduce the spatial dimensions, but this linear reduction is often too slow to counter the increased memory requirements due to network depth.

###### Pooling

In order to make working with big images feasible, we need techniques to reduce the spatial dimensions more strongly. This is where *pooling* layers prove very useful. A pooling layer reduces the spatial dimensions by combining a window of pixels to a single pixel. By sticking a pooling layer after every convolutional layer, the spatial dimensions are reduced exponentially, rather than linearly. This allows convolutional neural networks to process big chunks of data.

There are different ways to summarise multiple pixels into a single pixel. Two very common pooling techniques are

 1. **Average pooling** replaces the pixels by the mean intensity value in the window. 
 2. **Max pooling** replaces the pixels by the maximum intensity in the window.
 

###### Strides

In modern convolutional neural networks, *strided* or *dilated* convolutions (see visualisations below) are often preferred over pooling. With strided convolutions, the windows are shifted The main advantage of strided or dilated convolutions over pooling is that they can be learnt. This means that instead of relying on a fixed pooling technique, it is possible to effectively learn how the pixels in the window are to be summarised. Also note that average pooling can indeed be represented as a strided convolution with weights $\frac{1}{\text{window size}}$.

<div style="text-align: center">
  <figure style="display: inline-block; width: 49%;">
    <img style="padding: 46px 50px" src="https://raw.githubusercontent.com/vdumoulin/conv_arithmetic/master/gif/no_padding_strides.gif" />
    <figcaption style="width: 100%;"> Strided convolution </figcaption>
  </figure>
  <figure style="display: inline-block; width: 49%;">
    <img src="https://raw.githubusercontent.com/vdumoulin/conv_arithmetic/master/gif/dilation.gif" />
    <figcaption style="width: 100%; text-align: center;"> Dilated convolution </figcaption>
  </figure>
</div>

*visualisations taken from the [github repo](https://github.com/vdumoulin/conv_arithmetic) that comes with [this guide](https://arxiv.org/abs/1603.07285)*

### Exercise 5: Pooling (5 Points)

Since the `sig2col` function provides an array with the window elements in each column, it can also be used to implement pooling layers, when used correctly.

 > Implement the `MaxPool2d` module. You can use the `sig2col` function with its `stride` argument. You might also find the functions `np.take_along_axis` and `np.put_along_axis` useful.
 
**Hint:** You can apply `sig2col` on `np.arange(x.size).reshape(x.shape)` to obtain the indices of the input after the `sig2col` operation. This could be useful for implementing the back-propagation.

In [ ]:
class MaxPool2d(Module):
    """ Numpy DL implementation of a max-pooling layer. """

    def __init__(self, kernel_size):
        super().__init__()
        self.kernel_size = tuple(kernel_size)

    def compute_outputs(self, x):
        """
        Parameters
        ----------
        x : (N, C, H, W) ndarray

        Returns
        -------
        a : (N, C, H', W') ndarray
        cache : ndarray or tuple of ndarrays
        """
        # YOUR CODE HERE

        s = sig2col(x, self.kernel_size, stride=self.kernel_size) # shape is N, C, H', W', R1, R2
        s_flat = s.reshape(*s.shape[:-2], -1) # shape is N, C, H', W', R1 * R2 (last two dimensions are flattened)
        indices = s_flat.argmax(axis=-1, keepdims=True) # 
        cache = (np.array(x.shape), np.array(s_flat.shape), indices)
        a = np.take_along_axis(s_flat, indices, -1)[..., 0] # uses the indices along the last axis. Note that [..., 0] is the same as [:,:,:,:,0]
        return a, cache

    def compute_grads(self, grads, cache):
        """
        Parameters
        ----------
        grads : (N, C, H', W') ndarray
        cache : ndarray or tuple of ndarrays

        Returns
        -------
        dx : (N, C, H, W) ndarray
        """
        # YOUR CODE HERE
        x_shape, s_flat_shape, indices = cache
        s_grad_flat = np.zeros(s_flat_shape)
        dx = np.empty(x_shape) # has the same shape as x

        np.put_along_axis(s_grad_flat, indices, grads[..., np.newaxis], axis=-1) # put the gradients of the previous layer at the indices of the max values in the array s_grad_flat

        # now we need to bring this into the right shape such that we can put that into dx

        x_idx = np.arange(dx.size).reshape(x_shape) # like in the hint
        x_idx_s = sig2col(x_idx, self.kernel_size, stride=self.kernel_size) # like in the hint
        print(x_idx_s)
        print(x_idx_s.ravel())
        print(dx.size)
        print(s_grad_flat)
        dx.flat[x_idx_s.flatten()] = s_grad_flat.flatten()
        return dx

In [243]:
# Test Cell: do not edit or delete!
pooling = MaxPool2d((2, 3))
x = rng.standard_normal(size=(1, 1, 16, 18))
p, cache = pooling.compute_outputs(x)
assert isinstance(p, np.ndarray), (
    "ex5: output of MaxPool2d.compute_outputs is not a numpy array"
)
assert p.shape == x.shape[:2] + (8, 6), (
    "ex5: output of MaxPool2d.compute_outputs has incorrect shape"
)

In [244]:
# Test Cell: do not edit or delete!

In [245]:
# Test Cell: do not edit or delete!
g = pooling.compute_grads(np.ones_like(p), cache)
assert isinstance(g, np.ndarray), (
    "ex5: output of MaxPool2d.compute_grads is not a numpy array"
)
assert g.shape == x.shape, (
    "ex5: output of MaxPool2d.compute_grads has incorrect shape"
)

[[[[[[  0   1   2]
     [ 18  19  20]]

    [[  3   4   5]
     [ 21  22  23]]

    [[  6   7   8]
     [ 24  25  26]]

    [[  9  10  11]
     [ 27  28  29]]

    [[ 12  13  14]
     [ 30  31  32]]

    [[ 15  16  17]
     [ 33  34  35]]]


   [[[ 36  37  38]
     [ 54  55  56]]

    [[ 39  40  41]
     [ 57  58  59]]

    [[ 42  43  44]
     [ 60  61  62]]

    [[ 45  46  47]
     [ 63  64  65]]

    [[ 48  49  50]
     [ 66  67  68]]

    [[ 51  52  53]
     [ 69  70  71]]]


   [[[ 72  73  74]
     [ 90  91  92]]

    [[ 75  76  77]
     [ 93  94  95]]

    [[ 78  79  80]
     [ 96  97  98]]

    [[ 81  82  83]
     [ 99 100 101]]

    [[ 84  85  86]
     [102 103 104]]

    [[ 87  88  89]
     [105 106 107]]]


   [[[108 109 110]
     [126 127 128]]

    [[111 112 113]
     [129 130 131]]

    [[114 115 116]
     [132 133 134]]

    [[117 118 119]
     [135 136 137]]

    [[120 121 122]
     [138 139 140]]

    [[123 124 125]
     [141 142 143]]]


   [[[144 145 146]
     [162 163

In [246]:
# Test Cell: do not edit or delete!
assert gradient_check(pooling, x, debug=True), (
    "ex5: MaxPool2d module does not pass gradient check"
)

[[[[[[  0   1   2]
     [ 18  19  20]]

    [[  3   4   5]
     [ 21  22  23]]

    [[  6   7   8]
     [ 24  25  26]]

    [[  9  10  11]
     [ 27  28  29]]

    [[ 12  13  14]
     [ 30  31  32]]

    [[ 15  16  17]
     [ 33  34  35]]]


   [[[ 36  37  38]
     [ 54  55  56]]

    [[ 39  40  41]
     [ 57  58  59]]

    [[ 42  43  44]
     [ 60  61  62]]

    [[ 45  46  47]
     [ 63  64  65]]

    [[ 48  49  50]
     [ 66  67  68]]

    [[ 51  52  53]
     [ 69  70  71]]]


   [[[ 72  73  74]
     [ 90  91  92]]

    [[ 75  76  77]
     [ 93  94  95]]

    [[ 78  79  80]
     [ 96  97  98]]

    [[ 81  82  83]
     [ 99 100 101]]

    [[ 84  85  86]
     [102 103 104]]

    [[ 87  88  89]
     [105 106 107]]]


   [[[108 109 110]
     [126 127 128]]

    [[111 112 113]
     [129 130 131]]

    [[114 115 116]
     [132 133 134]]

    [[117 118 119]
     [135 136 137]]

    [[120 121 122]
     [138 139 140]]

    [[123 124 125]
     [141 142 143]]]


   [[[144 145 146]
     [162 163

In [247]:
# sanity check
pooling = MaxPool2d((2, 3))
pool_check = gradient_check(pooling, rng.standard_normal(size=(1, 1, 16, 18)), debug=True)
print("gradient check for MaxPool2D:", "passed" if pool_check else "failed")

[[[[[[  0   1   2]
     [ 18  19  20]]

    [[  3   4   5]
     [ 21  22  23]]

    [[  6   7   8]
     [ 24  25  26]]

    [[  9  10  11]
     [ 27  28  29]]

    [[ 12  13  14]
     [ 30  31  32]]

    [[ 15  16  17]
     [ 33  34  35]]]


   [[[ 36  37  38]
     [ 54  55  56]]

    [[ 39  40  41]
     [ 57  58  59]]

    [[ 42  43  44]
     [ 60  61  62]]

    [[ 45  46  47]
     [ 63  64  65]]

    [[ 48  49  50]
     [ 66  67  68]]

    [[ 51  52  53]
     [ 69  70  71]]]


   [[[ 72  73  74]
     [ 90  91  92]]

    [[ 75  76  77]
     [ 93  94  95]]

    [[ 78  79  80]
     [ 96  97  98]]

    [[ 81  82  83]
     [ 99 100 101]]

    [[ 84  85  86]
     [102 103 104]]

    [[ 87  88  89]
     [105 106 107]]]


   [[[108 109 110]
     [126 127 128]]

    [[111 112 113]
     [129 130 131]]

    [[114 115 116]
     [132 133 134]]

    [[117 118 119]
     [135 136 137]]

    [[120 121 122]
     [138 139 140]]

    [[123 124 125]
     [141 142 143]]]


   [[[144 145 146]
     [162 163